In [114]:
from sgp4.api import jday
import datetime
import pandas as pd
from sgp4.api import Satrec
from sgp4 import omm
import numpy as np
%matplotlib notebook
from sklearn.cluster import DBSCAN
import plotly.graph_objects as go
from sklearn.neighbors import KernelDensity
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans

In [115]:
#Get date
current_time = datetime.datetime.now()
xd, xr = jday(current_time.year , current_time.month , current_time.day, current_time.hour, current_time.minute , current_time.second)
print(xd)
print(xr)

2460402.5
0.8530092592592593


In [116]:
with open('../Data/satellitedata.csv') as f:
    lines = omm.parse_csv(f)
    fields = [field for field in lines]
 

In [117]:
Satellite = []
result_satellite = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    if any([abs(coord) > 10_000 for coord in r]):
     continue
    Satellite.append(r)
    result_satellite.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])
    
    
  

data_satellite = np.array(Satellite)
data_satellite = data_satellite[~np.isnan(data_satellite).any(axis=1)]

result_satellite = pd.DataFrame(result_satellite,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()

fig = go.Figure(data=[go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2],
                                   mode='markers',name="Satellite", marker={"size":12, "color":"white"})])

fig.show()

In [118]:
normalization = np.linalg.norm(data_satellite, axis=1)[:, np.newaxis]
data_satellite_norm = data_satellite / normalization

fig = go.Figure(data=[go.Scatter3d(x=data_satellite_norm[:,0], y=data_satellite_norm[:,1], z=data_satellite_norm[:,2],
                                   mode='markers',name="Satellite", marker={"size":10, "color":"white"})])

fig.show()

In [119]:
# DBSCAN for cosmos garbage
with open('../Data/cosmos.csv') as f:
        lines = omm.parse_csv(f)
        fields = [field for field in lines]

Cosmos_G = []
result_cosmos = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    Cosmos_G.append(r)
    result_cosmos.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])

data_garbage_cosmos = np.array(Cosmos_G)
data_garbage_cosmos = data_garbage_cosmos[~np.isnan(data_garbage_cosmos).any(axis=1)]

result_cosmos = pd.DataFrame(result_cosmos,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers')])
fig.show()


model = DBSCAN(eps=800, min_samples=4)
model.fit_predict(data_garbage_cosmos)
pred_cosmos = model.fit_predict(data_garbage_cosmos)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":pred_cosmos})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'gold'}))

fig.show()

print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)


number of cluster found: 42
cluster for each point:  [-1  0 -1  1 -1 -1  0  1  2  3 -1 13 -1 -1 -1 -1 -1  4 -1 -1  1  5  6  1
  1 -1  7 -1 -1 -1 -1 -1  1 -1 -1  1 32 -1 -1 -1 16  8  9 14  1 10 -1 -1
 -1  2 -1 11 22 12 10 -1  1 38  1 -1 -1  6 -1 -1  2 13  1  1 -1  1  5 -1
 27  1  5 -1 -1 19  5 -1 14 -1 24 -1 -1 -1  2 15 16 15  1 -1 31  8 16  5
 -1 17 -1 -1 18 -1 -1 -1 18 -1  1  1 -1 -1 34  1 19  4  2  2  2 13  1 -1
 -1 -1 20 -1  1  2 23  1  5 21 -1 22 -1 23  5  0 20 -1 -1  1  5 -1  1  5
 -1 -1 10  8  1  2 -1 24  5 -1 24 -1 -1  1 -1 -1 -1 -1  1 18 -1  1 -1  8
  1  1  3 -1 18  1 -1  7 -1  5 -1  1  5 17 -1  8 25  1 11 12  2 -1 -1  8
  3 -1 39 28  1  1 26 -1  5 -1 -1 -1 -1  5 -1  1  1 -1  8 -1 22 -1  4  5
 -1  5 -1  8 -1  7 -1 39 -1  5 -1  2 -1 -1 34 -1 -1 32  1 27  1  2  1  3
 -1 28  5 -1 -1  5  1  9 -1 29  6 -1 -1  1 -1 18 -1 -1 -1  5 -1  5 -1 -1
  5  5  5 37 -1  8 -1  5  1  1 30 -1 -1  2 24  3 31 -1  5 -1  1 27 -1 -1
  1  1 23 -1  8 -1  1 -1  1 -1  5  1 32 20  2 10  5  1  5  1  5  1 -1  

In [120]:
# DBSCAN for iridium garbage
with open('../Data/iridium.csv') as f:
        lines = omm.parse_csv(f)
        fields = [field for field in lines]

Iridium_G = []
result_iridium = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    Iridium_G.append(r)
    result_iridium.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])
    
    

data_garbage_iridium = np.array(Iridium_G)
data_garbage_iridium = data_garbage_iridium[~np.isnan(data_garbage_iridium).any(axis=1)]


result_iridium = pd.DataFrame(result_iridium,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()


fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers')])
fig.show()

model = DBSCAN(eps=1570, min_samples=4)
model.fit_predict(data_garbage_iridium)
pred_iridium = model.fit_predict(data_garbage_iridium)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":12, "color":pred_iridium})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'gold'}))

fig.show()

print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)

number of cluster found: 10
cluster for each point:  [-1  0  1  2 -1  1  3  4  1  4  0 -1  2  0  5  2  4  0 -1  1  0 -1  0 -1
  0  0 -1  3  5  2  5  2  4  4  0  0 -1  6  2  6  3  4 -1  2  2  3  0  0
  0  7 -1 -1  2 -1 -1  2  1  2  4  0  1 -1 -1  2  2  3 -1 -1 -1 -1  4 -1
  0  1 -1  0 -1  7 -1 -1  1  4 -1 -1  7 -1  8 -1  2 -1 -1 -1 -1  2  2  2
 -1  2  0  2 -1  1  8  4  7  2  7  3 -1 -1  1  0 -1  2  4  3 -1 -1  7 -1
 -1  0 -1 -1 -1  2  4  4 -1  2 -1  6  4  8 -1 -1  2  6 -1  4 -1  1  2  4
  2  2  2 -1 -1  2  1 -1  6 -1  8  4  1  2  5 -1 -1  2 -1  2  0  0  0  1
  2 -1]


In [121]:
#KDE for iridium garbage
model = KernelDensity(kernel='gaussian', bandwidth = 1500)
model.fit(data_garbage_iridium)
log_dens_iridium = model.score_samples(data_garbage_iridium)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":log_dens_iridium})])
fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'white'}))
fig.show()

In [122]:
#KDE for cosmos garbage
model = KernelDensity(kernel='gaussian', bandwidth = 1000)
model.fit(data_garbage_cosmos)
log_dens_cosmos = model.score_samples(data_garbage_cosmos)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":log_dens_cosmos})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'white'}))
fig.show()


In [123]:
# EM Algorithm for data_garbage_cosmos


# Filter out rows with inf or NaN values
data_garbage_cosmos = np.array([row for row in data_garbage_cosmos if np.isfinite(row).all()])

# Initialize parameters
n_components = 3
mu = np.random.rand(n_components, 3)  # Random initialization of means
sigma = [np.eye(3) for _ in range(n_components)]  # Initialization of covariance matrices as identity matrices
pi = np.ones(n_components) / n_components  # Initialization of mixture weights as a uniform distribution
z = np.zeros((len(data_garbage_cosmos), n_components))

def likelihood(x, mu, sigma):
    sigma = np.where(np.isinf(sigma) | np.isnan(sigma), 1e-6 * np.eye(3), sigma)
    return multivariate_normal(mean=mu, cov=sigma).pdf(x) + np.finfo(float).eps

# EM algorithm
for step in range(100):
    # E-step
    for k in range(n_components):
        z[:, k] = pi[k] * likelihood(data_garbage_cosmos, mu[k], sigma[k])
        z /= z.sum(axis=1, keepdims=True) + np.finfo(float).eps

    # M-step
    for k in range(n_components):
        weight = np.sum(z[:, k])
        mu[k] = np.sum(z[:, k, np.newaxis] * data_garbage_cosmos, axis=0) / weight
        diff = data_garbage_cosmos - mu[k]
        cov = np.dot(diff.T, z[:, k, np.newaxis] * diff) / weight
        cov += 1e-3 * np.eye(3)  # Add small noise to covariance matrix to avoid numerical instabilities
        sigma[k] = cov
        pi[k] = np.clip((weight + 1e-6) / (len(data_garbage_cosmos) + n_components * 1e-6), 0, 1)

# Visualization
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:, 0], y=data_garbage_cosmos[:, 1], z=data_garbage_cosmos[:, 2], mode='markers', name='Data')])
fig.add_trace(go.Scatter3d(x=mu[:, 0], y=mu[:, 1], z=mu[:, 2], mode='markers', marker=dict(size=8), name='Means'))
fig.show()

In [124]:
# EM Algorithm for data_garbage_iridium


# Filter out rows with inf or NaN values
data_garbage_iridium = np.array([row for row in data_garbage_iridium if np.isfinite(row).all()])

# Initialize parameters
n_components = 3
mu = np.random.rand(n_components, 3)  # Random initialization of means
sigma = [np.eye(3) for _ in range(n_components)]  # Initialization of covariance matrices as identity matrices
pi = np.ones(n_components) / n_components  # Initialization of mixture weights as a uniform distribution
z = np.zeros((len(data_garbage_iridium), n_components))

def likelihood(x, mu, sigma):
    sigma = np.where(np.isinf(sigma) | np.isnan(sigma), 1e-6 * np.eye(3), sigma)
    return multivariate_normal(mean=mu, cov=sigma).pdf(x) + np.finfo(float).eps

# EM algorithm
for step in range(100):
    # E-step
    for k in range(n_components):
        z[:, k] = pi[k] * likelihood(data_garbage_iridium, mu[k], sigma[k])
        z /= z.sum(axis=1, keepdims=True) + np.finfo(float).eps

    # M-step
    for k in range(n_components):
        weight = np.sum(z[:, k])
        mu[k] = np.sum(z[:, k, np.newaxis] * data_garbage_iridium, axis=0) / weight
        diff = data_garbage_iridium - mu[k]
        cov = np.dot(diff.T, z[:, k, np.newaxis] * diff) / weight
        cov += 1e-3 * np.eye(3)  # Add small noise to covariance matrix to avoid numerical instabilities
        sigma[k] = cov
        pi[k] = np.clip((weight + 1e-6) / (len(data_garbage_iridium) + n_components * 1e-6), 0, 1)

# Visualization
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:, 0], y=data_garbage_iridium[:, 1], z=data_garbage_iridium[:, 2], mode='markers', name='Data')])
fig.add_trace(go.Scatter3d(x=mu[:, 0], y=mu[:, 1], z=mu[:, 2], mode='markers', marker=dict(size=8), name='Means'))
fig.show()

In [125]:
#K-Means for data_garbage_cosmos
kmeans = KMeans(n_clusters=20)
kmeans.fit(data_garbage_cosmos)

labels = kmeans.labels_

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:, 0], y=data_garbage_cosmos[:, 1], z=data_garbage_cosmos[:, 2], mode='markers', name='Data', marker=dict(size=8, color=labels, colorscale='Viridis'))])

fig.add_trace(go.Scatter3d(x=kmeans.cluster_centers_[:, 0], y=kmeans.cluster_centers_[:, 1], z=kmeans.cluster_centers_[:, 2],
                          mode='markers', name='Cluster Centers', marker=dict(size=12, color='red', opacity=0.7)))

fig.show()

In [126]:
#K-means for data
kmeans = KMeans(n_clusters=20)
kmeans.fit(data_garbage_iridium)

labels = kmeans.labels_
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":12, "color":labels})])

fig.add_trace(go.Scatter3d(x=kmeans.cluster_centers_[:, 0], y=kmeans.cluster_centers_[:, 1], z=kmeans.cluster_centers_[:, 2],
                          mode='markers', name='Cluster Centers', marker=dict(size=12, color='red', opacity=0.7)))

fig.show()

In [127]:
result_iridium['Cluster'] = pred_iridium
result_iridium['Density'] = log_dens_iridium
result_iridium.to_csv('../Data/result_iridium.csv', index= False)
result_iridium

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z,Cluster,Density
0,IRIDIUM 33,1997-051C,2024-03-15T03:57:12.554496,4111.708312,-2970.049456,-5060.235166,-1,-28.941383
1,IRIDIUM 33 DEB,1997-051L,2024-03-14T17:47:59.202816,3945.025833,-2635.167238,5310.811652,0,-28.213342
2,IRIDIUM 33 DEB,1997-051N,2024-03-15T04:39:16.727904,-4855.565652,5245.088670,319.048151,1,-28.125552
3,IRIDIUM 33 DEB,1997-051P,2024-03-14T20:44:27.452832,-2369.858627,405.002167,-6753.589167,2,-27.728935
4,IRIDIUM 33 DEB,1997-051Q,2024-03-14T15:11:01.815360,3849.898839,-5970.939986,481.196198,-1,-28.294474
...,...,...,...,...,...,...,...,...
165,IRIDIUM 33 DEB,1997-051ABX,2024-03-14T23:53:25.857024,-1308.997181,-5370.915332,4463.127462,0,-28.329313
166,IRIDIUM 33 DEB,1997-051ACD,2024-03-14T07:15:03.155904,-702.476451,-4105.024004,5703.803950,0,-27.952053
167,IRIDIUM 33 DEB,1997-051ACE,2024-03-11T22:52:08.279904,-4074.542707,5794.859689,-817.050095,1,-27.973849
168,IRIDIUM 33 DEB,1997-051ACG,2024-03-14T13:33:50.161824,-2848.170405,-990.073967,-6622.411745,2,-28.007994


In [128]:
result_cosmos['Cluster'] = pred_cosmos
result_cosmos['Density'] = log_dens_cosmos
result_cosmos.to_csv('../Data/result_cosmos.csv', index= False)
result_cosmos

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z,Cluster,Density
0,COSMOS 2251,1993-036A,2024-03-14T21:31:51.966336,-7111.098260,-779.659541,59.786965,-1,-28.818937
1,COSMOS 2251 DEB,1993-036E,2024-03-14T21:50:15.684000,-2791.166773,1246.943377,6466.245178,0,-27.435453
2,COSMOS 2251 DEB,1993-036F,2024-03-14T18:05:33.239616,-10.012171,-4064.685839,-5874.198316,-1,-28.001112
3,COSMOS 2251 DEB,1993-036G,2024-03-14T08:58:39.399168,-2887.633166,-58.520907,6405.275049,1,-27.634856
4,COSMOS 2251 DEB,1993-036H,2024-03-14T21:23:40.216416,5170.237921,-695.838754,4911.655067,-1,-27.531678
...,...,...,...,...,...,...,...,...
769,COSMOS 2251 DEB,1993-036BXS,2024-03-04T05:57:55.018944,6926.331522,2296.158668,904.184180,27,-27.911711
770,COSMOS 2251 DEB,1993-036BXY,2024-03-14T01:11:36.902688,1589.060704,-6889.846422,-997.375792,11,-28.420698
771,COSMOS 2251 DEB,1993-036BYA,2024-03-14T03:53:33.007776,2413.698325,1960.237469,-6332.897713,5,-27.170609
772,COSMOS 2251 DEB,1993-036BYD,2024-03-09T04:50:01.458528,5168.560423,2887.071160,-4419.262254,26,-27.833399


In [129]:
result_satellite.to_csv('../Data/result_satellite.csv', index= False)
result_satellite

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z
0,HBTSS-SV2,2024-028A,2024-03-14T20:49:05.202912,4710.274031,-5518.608659,1339.912252
1,RAPTOR4,2024-028B,2024-03-14T20:47:39.831936,5329.545390,-5051.717024,713.276411
2,RAPTOR1,2024-028C,2024-03-14T20:46:13.850112,5875.432534,-4461.844891,41.959327
3,RAPTOR3,2024-028D,2024-03-14T22:29:38.631552,6328.304303,-3730.837655,-678.650830
4,RAPTOR2,2024-028E,2024-03-14T20:43:13.249056,6625.147992,-2949.251756,-1353.426841
...,...,...,...,...,...,...
198,STARLINK-31289,2024-044X,2024-03-15T04:00:00.999936,1094.676246,5089.534048,-3714.056342
199,STARLINK-31283,2024-044Y,2024-03-15T04:00:00.999936,-2981.756449,5549.493223,-978.673702
200,EXO-0,2023-174DK,2024-03-14T21:03:32.576832,3958.636969,6.605955,5631.460168
201,2024-047A,2024-047A,2024-03-15T04:34:59.235168,-436.699709,1129.559523,-6832.819509


In [130]:
result_iridium.groupby('Cluster')['Density'].mean().abs()

Cluster
-1    28.605563
 0    27.872823
 1    28.026901
 2    27.685242
 3    28.162327
 4    27.714398
 5    28.328414
 6    27.908610
 7    28.364403
 8    27.969858
Name: Density, dtype: float64

In [131]:
result_cosmos.groupby('Cluster')['Density'].mean().abs()

Cluster
-1     28.294999
 0     27.550553
 1     27.375148
 2     27.587647
 3     27.625404
 4     28.064415
 5     27.323315
 6     27.755101
 7     28.534721
 8     27.646587
 9     28.067870
 10    27.860769
 11    28.339708
 12    27.971182
 13    27.831218
 14    27.730565
 15    27.928085
 16    28.164863
 17    27.608777
 18    27.974892
 19    28.228344
 20    27.673407
 21    28.183727
 22    28.065590
 23    28.167480
 24    27.990103
 25    28.077131
 26    27.684954
 27    27.909169
 28    28.216804
 29    27.989062
 30    28.126931
 31    27.796356
 32    27.993640
 33    28.370665
 34    27.960073
 35    27.946137
 36    28.026216
 37    28.195690
 38    28.327534
 39    28.266039
 40    27.944677
Name: Density, dtype: float64

In [132]:
distances_s_i = []
for i, garbage in enumerate(data_garbage_iridium):
    for j, satellite in enumerate(data_satellite):
        distance = np.sqrt((garbage[0] - satellite[0])**2 + (garbage[1] - satellite[1])**2 + (garbage[2] - satellite[2])**2)
        distances_s_i.append((i, j, distance))

distances = pd.DataFrame(distances_s_i, columns=['garbage_index', 'satellite_index', 'distance'])
print(distances)

distances.to_csv('../Data/distances_s_i.csv', index=False)

       garbage_index  satellite_index      distance
0                  0                0   6914.862392
1                  0                1   6256.988328
2                  0                2   5600.764715
3                  0                3   4968.941311
4                  0                4   4478.642599
...              ...              ...           ...
34505            169              198   6739.244056
34506            169              199   3687.095483
34507            169              200  13141.770910
34508            169              201   6789.046550
34509            169              202   7933.547494

[34510 rows x 3 columns]


In [133]:
distances_s_c = []
for i, garbage in enumerate(data_garbage_cosmos):
    for j, satellite in enumerate(data_satellite):
        distance = np.sqrt((garbage[0] - satellite[0])**2 + (garbage[1] - satellite[1])**2 + (garbage[2] - satellite[2])**2)
        distances_s_c.append((i, j, distance))

distances = pd.DataFrame(distances_s_c, columns=['garbage_index', 'satellite_index', 'distance'])
print(distances)

distances.to_csv('../Data/distances_s_c.csv', index=False)

        garbage_index  satellite_index      distance
0                   0                0  12800.046966
1                   0                1  13169.933106
2                   0                2  13498.473574
3                   0                3  13779.415222
4                   0                4  13978.153123
...               ...              ...           ...
157117            773              198  12058.169768
157118            773              199   8895.937474
157119            773              200   8481.685084
157120            773              201  13007.257962
157121            773              202  12776.110990

[157122 rows x 3 columns]
